# Tutorial 5 - Classification using Random Forest
In this last tutorial, we will look at the estimation of solar radiation as a *classification* task rather than a *regression* task. Let's assume that we are not interested in knowing the exact solar radiation that can be received on a rooftop, but rather we want to know if the roof has a very good, good, average, bad or very bad solar yield.

To obtain the class labels, we have hence mapped the `tilted_radiation` ($G_t$) from tutorials 1-4 based on the following class:
- $G_t > 1400$ kWh/m$^2$ --> Very good
- $1200 < G_t < 1400$ kWh/m$^2$ --> Good
- $1000 < G_t < 1200$ kWh/m$^2$ --> Average
- $800 < G_t < 1000$ kWh/m$^2$ --> Bad
- $G_t < 800$ kWh/m$^2$ --> Very bad

These classes will **replace** the continuous tilted radiation target in this tutorial. The feature set hence remains the same, so we can focus on some modelling specifics of classification problems. These include:
- Class probability and majority voting
- Error metrics for classification (accuracy, f1-score, recall, etc.)
- Confusion matrices

In this tutorial, we will use simple validation to demonstrate the concepts of classification, which differ from the regression tasks we have seen before. The main workflow (fitting, prediction, cross-validation, etc.) is identical to regression tasks which we have seen in Tutorials 1-4. In the final task (3.2), you will combine the concepts you have learnt throughout this and the previous tutorial series and perform the full tuning of a classification forest yourself.

# Part 1: Load and pre-process data


As seen in the previous tutorials, we will load the preprocessed data. Additionally, we will **replace the target variable** by the classes shown above (Step 1.2). 

In Step 1.3, we will have a brief look at the distribution of the target classes and classification measures for feature selection. In Step 1.4, we split the data into training and validation data for simple validation.

In [ ]:
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

from lib_file import *  # Contains the function which generates the classification target

## Step 1.1: Load the data

In [ ]:
data = ## TASK: Load the data from the correct file

## Step 1.2: Obtain the class labels

To obtain the radiation class, we have provided you with the function `map_radiation_class` in the `lib_file.py` (loaded above). You can apply the function such as:

In [ ]:
data['radiation_class'] = map_radiation_class(data.tilted_radiation)

**_TASK_**: Compare the two columns of `data`, namely the tilted radiation and the classes, to verify that the classification has been correctly applied.

Now we can extract the features (`X`) and the new target (`y`) from the data:

**IMPORTANT**: Do not use the `tilted_radiation` as a feature in the model, this would be cheating ;) We are simulating here a case where we *do not know* the tilted radiation, but are interested only in a classification and not in precise values.

In [ ]:
target_name   = 'radiation_class'
features_list = [ 'roof_tilt', 'roof_aspect', 'roof_area', 'roof_x', 'roof_y', 'SIS', 'aspect_sign',
                  'SISDIR', 'DHI', 'horizon_S', 'horizon_SSW', 'horizon_SWW', 'horizon_W',
                  'horizon_NWW', 'horizon_SSE', 'horizon_SEE', 'horizon_E', 'horizon_NEE', ]

In [ ]:
y = # TASK: EXTRACT TARGET
X = # TASK: EXTRACT FEATURES

## Step 1.3: Analyse new target

As the target has changed, despite the feature set remaining unchanged, we should check the following two aspects:
1. The distribution of the classes in the target
2. The dependencies between the target and the features

As the classification target is a *discrete* and not a *continuous* variable, the correlation coefficient is not applicable. Instead, we can perform a $\chi ^2$-Test (chi2), which however can only be performed with positive features. Alternatively, a function `f_classif` has been implmented in `scikit-learn` for linear dependency, and `mutual_info_classif` for any statistical dependency.

For more information, see: https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection

Let's load the mutual information for classification, as well as the `matplotlib` library to visualise the class distribution:

In [ ]:
from sklearn.feature_selection import mutual_info_classif

%matplotlib notebook
import matplotlib.pyplot as plt

##### 1. Check class distribution
The class distribution is best visualised through a **histogram** (bar chart). For this, let's obtain the number of samples in each class: 

In [ ]:
# Get the number of samples for each class
hist = y.value_counts()
hist

As you can see, the classes are not displayed in the right order. To sort them according to their "logic" sequence, we can apply a little trick:

In [ ]:
classes     = ['Very bad', 'Bad', 'Average', 'Good', 'Very good']  # List of classes in logical order
hist_sorted = hist.loc[classes]  # Re-order column by the classes-list
hist_sorted

**TASK**: Visualise the distribution of the target classes as a bar chart.

## *Questions* 
- a. Are the classes well-balanced? How do you identify that?
- b. Can you think of some problems if classes are not well-balanced?

##### 2. Dependencies between features and targets
To develop an understanding for the dependencies between the features (`X`) and the target (`y`), we will compare the mututal information:

In [ ]:
mutual_info_df = # TASK: Create a Series with mutual information between X and y (with name 'Mutual_info')
# TASK: Display mutual_info_df in descending order

## *Question*
- How does the mutual information compare to that of the regression task (Tutorial 2)?

**NOTE**: Below, we will continue to use all features, as the Random Forest's performance does not deteriorate significantly with a larger number of features, and as this allows us to assess, for example, the feature importance. A feature selection can, however, be used in the model tuning in Part 3.2.

# Part 2 - Random Forest Classifiers

The random forest classifier works much like the random forest regressor, with two main differences:

1. Splits are performed based on the **gini impurity** (or entropy)
2. Predictions are obtained by **majority voting**

In this part, we will see how the defaul Random Forest functions (the principal hyper-parameters are the same). We will see that classification forests can also be used to obtain class *probabilities*, and we will look into the visualisation of a classification tree and the feature importance.

Documentation of the RF classifier: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
# TASK: Import the Random Forest Classifier

## Step 2.1: Model fitting and prediction

For the simple validation, split the data into training and validation set before fitting any ML model:

In [ ]:
X_train, X_val, y_train, y_val = ## TASK: SPLIT THE DATA into 80% training and 20% validation data

With the Random Forest Classifier imported and the data splitted into training and test set, let's declare a default forest, fit it with the training data and obtain the prediction for the validation set:

In [ ]:
rf = # TASK: Declare default RF
# Fit with training data

In [ ]:
y_pred = # TASK: Predict for the validation set

Let's see how the predictions correspond to the target labels:

**TASK**: Create a dataframe called `comparison` with two columns - the target radiation class and the prediction and display the first 5 lines.

## Step 2.2: Predict class probabilities
The advantage of decision trees and random forests over other algorithms such as SVM is that due to the collection of samples in the leaves of the trees, the algorithm yields **class probabilities**, which can help to determine the *confidence* of a given prediction. These probabilities can be particularly useful if the random forest is used for decision making.

**NOTE**: The class probabilities of individual trees can be misleading if a tree is fully grown (over-fitted), since the probability is always 1 if only 1 sample is present in a leaf. This effect is averaged out in a forest, as probabilities are aggregated from all trees in the forest.

Class probabilities are predicted as:

In [ ]:
y_pred_proba = rf.predict_proba(X_val)

Now let's check the output `y_pred_proba`:

In [ ]:
y_pred_proba

And its shape:

In [ ]:
# TASK: Compute the shape of y_pred_proba

As you can see, the output has 5 columns, which correspond to the 5 targets. To give each column a name, we can use the classes attribute of the Random Forest, namely the `rf.classes_`, which contains the order of the class labels assigned by the RF:

In [ ]:
rf.classes_

**TASK**: Concatenate the `comparison` dataframe from above with the class probabilities and display this dataframe.

*HINT 1*: Your output dataframe should have 7 columns.<br>
*HINT 2*: Don't forget to give the classes their correct name.<br>
*HINT 3*: Use the "trick" from step 1.3.1 to display the columns (classes) in their "logical" order.

## *Questions*
- a. Do the predictions corrrespond to the target?
- b. What are the class probabilities in the cases of mis-classification?
- c. How are the class labels sorted in the `rf.classes_`-attribute? What could you do to obtain a default "logical" order of the classes? (from Very bad to Very good)

## Step 2.3: Visualise classification tree

As for regression, we can visualise the classification tree, in order to "look inside" its functioning. We will visualise the first tree of the forest here, to see the differences to the regression tree from Tutorial 4.

In [ ]:
from sklearn.tree import export_graphviz 
from IPython.display import Image

Let's extract the first tree of the forest. The trees are stored as a list in the `rf.estimators_` attribute.

In [ ]:
decision_tree = # TASK: Extract the first element of the list stored in the rf.estimators_ attribute

[OPTIONAL] Following Step 2.2 in Tutorial 4, we can export the first three layers of the tree as a .dot file, and generate a png from it.

In [ ]:
export_graphviz(decision_tree, max_depth = 3, out_file="tree_class.dot", feature_names=features_list, filled=True)

In [ ]:
# [OPTIONAL - see Tutorial 4 Step 2.2]
# Make sure you installed graphviz (exclamation mark is for shell commands)
# Convert dot file to png file.
!dot -Tpng tree_class.dot -o tree_class.png

**NOTE**: If you have not installed graphviz, the above task will fail and you will load the tree_class.png that we have provided. This was generated from a different tree and a different train_test_split than your current notebook, but it is based on the same dataset.

This png can be visualised as:

In [ ]:
Image(filename='tree_class.png') 

## *Questions*
- a. What are the main differences to the regression tree?
- b. What do the entries in the "value" array correspond to? 
- c. What do the colors represent?

## Step 2.4 Feature importance

As for the regression forest, we can compute and visualise the feature importance and its standard deviation.

**TASK**: Compute and visualise the feature importance with its standard deviation of the fitted Random Forest.

## *Questions*
- a. What are the differences to the regression feature importance?
- b. Are there discrepancies with the mutual information?

# Part 3 - Evaluating classification results

In Part 2, we have looked at *what* a classification forest does and *how* it does it, but we have not yet looked into *how well* it performs. How do we know what defines a "good" or a "bad" classification? In classification tasks, the error metrics we have seen in the previous tutorial ($R^2$, MAE etc.) are no longer applicable since the target is now _discrete_.

For discrete output variables, performance metrics represent different statistics of the number of correct and incorrect predictions. From these, we can derive several metrics:

1. **Accuracy**: All correct predictions / all predictions
2. **Precision**: True positives / all predicted positives
3. **Recall**: True positives / all actual positives (target positives)
4. **F1-Score**: Harmonic mean of precision and recall 

In the case of binary classification (only *Positive* and *Negative* classes) correct predictions are referred to as **true positives** (correctly classified *Positive*) & **true negatives** (correctly classified *Negative*), and incorrect predictions as **false positives** (classified as *Positive* but actually *Negative*) & **false negatives** (classified as *Negative* but actually *Positive*). 

In the case of multi-class classification, like we have it in our example, this principle functions very similarly. The metrics (except accuracy) are computed separately for each class, by considering this class as the *Positive* class and the sum of all other classes as *Negative* (we will see an example below). 

For more explanation of the classification scores, see this link: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9 <br>or the Wikipedia entry: https://en.wikipedia.org/wiki/Precision_and_recall

## *Questions*
- a. Which is the default metric to score classification tasks in `scikit-learn`? 
- b. What is the value of this score for the fitted forest from Part 2?
- c. In which case can the default metric be misleading?

## Step 3.1: Evaluation metrics
We can use two tools which help us to evaluate the classification results, namely the `confusion_matrix`, which shows all correct and incorrect classifications, and the `classification_report`, which computes all metrics mentioned above:

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

##### 1. Confusion matrix
The confusion matrix is a matrix with all "True" (target) classes as rows, and all "Predicted" classes as columns. It gives an overview of the number of correct and incorrect classifications, and is the basis of the calculation of accuracy, precision, recall and F1-score.

The confusion matrix itself is a simple 2D-Array, but with the knowledge of the RF class assignment, we can transform it into a dataframe:

In [ ]:
conf_matrix = # TASK: Obtain confusion matrix for the validation data

In [ ]:
confusion_df = pd.DataFrame( conf_matrix,                                    # Create dataframe
                             index=rf.classes_, columns=rf.classes_          # Assign class names to columns and rows
                           ).loc[classes,classes]                            # Re-order classes logically

# Add "True" & "Predicted" labels
confusion_df = pd.concat({'Predicted': pd.concat({'True': confusion_df})}, axis = 1)  

In [ ]:
# TASK: Display confusion matrix (dataframe)

## *Questions*
- a. How do you compute the **accuracy** by using the confusion matrix? Does the computation yield the value you expect?
- b. How do you compute the **precision** of the "Average" class from the confusion matrix? What is the value for your data?
- c. How do you compute the **recall** of the "Average" class from the confusion matrix? What is the value for your data?

**NOTE**: You can do these calculations either by hand or by code.

##### 2. Classification report
In practice, we obviously do not want to compute all scores for all classes by hand (but for interpreting these scores correctly, we need to know how they have been obtained). Instead, we can compute the classification report and print the results, which performs this task for us:

In [ ]:
print(classification_report(y_val, y_pred, target_names=rf.classes_))

## *Questions*
- a. What is the "support" of the classification report?
- b. What is the difference between the `marco avg` and the `weighted avg`? 
- c. For which class does the classifier have the highest precision? How does the precision compare to the recall? Can you explain this difference?
- d. For which class does the algorithm perform overall the best? 

## [OPTIONAL] Step 3.2: Model tuning 

Now it is your turn: Applying the tools that we have seen throughout this tutorial series, you can now tune the Random Forest classifier to optimize its performance for the given classification task.

**TASK**: Tune the RF classifier to improve the model performance (using cross-validation). 

**NOTE**: You can also consider feature selection or other ML models as tools to maximise model performance.

## *Questions*
- a. What is the best validation accuracy that you can achieve? Which are the hyper-parameters to do so?
- b. What are the values of the accuracy, the F1-score and the confusion matrix for the test set?